<a href="https://colab.research.google.com/github/leo-breda/HMWK1_ANN2DL/blob/main/data_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/My Drive/HMK1

/gdrive/My Drive/HMK1


In [ ]:
from google.colab import drive
import tensorflow as tf
import numpy as np
import os
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

from keras import backend as K
from PIL import Image

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)


2.10.1


In [ ]:
# Random seed
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Data merging
## one-time use only!

In [ ]:
dataset_dir = 'data/original_data'
# Plot example images from dataset
labels = ['Species1',       # 0
          'Species2',       # 1
          'Species3',       # 2
          'Species4',       # 3
          'Species5',       # 4
          'Species6',       # 5
          'Species7',       # 6
          'Species8'        # 7
]


def folderToPaths(full_img_dir):

    x_paths_list = []

    full_img_dir = full_img_dir

    for full in os.listdir(full_img_dir):
         x_paths_list.append(os.path.join(full_img_dir, full))
    
    x_paths_list.sort()
    return x_paths_list


In [ ]:
# mergin all the images, one time use only!
for idx, class_label in enumerate(labels) :
  paths = folderToPaths(full_img_dir = '{}/{}/'.format(dataset_dir, class_label))
  
  for path in paths:
    s = str(idx) + "_" + path[-9:]
    new_path = os.path.join(merged_dataset_dir, s)
    shutil.copy(path, new_path)

# Data splitting

In [ ]:
merged_dataset_dir = "data/merged_data"
paths_merged = folderToPaths(merged_dataset_dir)
labels_of_paths = [p[-11] for p in paths_merged] # get all labels from the image path

validation_percentage = 0.15
test_percentage = 0.2

X_train_val, X_test = train_test_split(paths_merged, test_size = test_percentage, shuffle = True, stratify = labels_of_paths)

labels_of_paths_train_val = [p[-11] for p in X_train_val]

X_train, X_val = train_test_split(X_train_val, test_size = validation_percentage, shuffle = True, stratify = labels_of_paths_train_val)

labels_train = [p[-11] for p in X_train]
labels_test = [p[-11] for p in X_test]
labels_val =  [p[-11] for p in X_val]

# Moving the file into the folders

In [ ]:
import shutil
from tqdm import tqdm
import glob


for root, subdirectories, fi in os.walk("data/leo/test/"):
    for sub in subdirectories:
        files = os.listdir(root+sub)
        for f in files:
            os.remove(root+sub+"/"+f)
for root, subdirectories, fi in os.walk("data/leo/train/"):
    for sub in subdirectories:
        files = os.listdir(root+sub)
        for f in files:
            os.remove(root+sub+"/"+f)
for root, subdirectories, fi in os.walk("data/leo/val/"):
    for sub in subdirectories:
        files = os.listdir(root+sub)
        for f in files:
            os.remove(root+sub+"/"+f)

for img in tqdm(zip(labels_train,X_train),total=len(labels_train)):
  shutil.copy(img[1],"data/leo/train/"+str(int(img[0])+1))

for img in tqdm(zip(labels_test,X_test),total=len(labels_test)):
  shutil.copy(img[1],"data/leo/test/"+str(int(img[0])+1))

for img in tqdm(zip(labels_val,X_val),total=len(labels_val)):
  shutil.copy(img[1],"data/leo/val/"+str(int(img[0])+1))


# Keras data loader

In [ ]:
training_dir = "data/leo/train"
validation_dir = "data/leo/val"
test_dir = "data/leo/test"

train_data = tf.keras.utils.image_dataset_from_directory(
    training_dir,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=32,
    image_size=(96, 96),
    shuffle=True,
    seed=seed)

val_data = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=32,
    image_size=(96, 96),
    shuffle=True,
    seed=seed)

test_data = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=32,
    image_size=(96, 96),
    shuffle=True,
    seed=seed)


Found 2265 files belonging to 8 classes.
Found 567 files belonging to 8 classes.
Found 709 files belonging to 8 classes.


# mock model with Preprocessing and Augmentation

In [ ]:
import keras_cv #for CutOut agumentation

inputs = tfk.Input((96,96,3))

# RESIZING
x = tfkl.Resizing(224,224,interpolation = "bicubic")(inputs)

# AUGMENTATION
x =  keras_cv.layers.RandomCutout(0.35, 0.35)(x)

x = tf.keras.layers.RandomBrightness(
    0.25, value_range=(0, 255), seed=seed
)(x)

x= tf.keras.layers.RandomFlip(
    mode="horizontal_and_vertical", seed=seed
)(x)

x = tf.keras.layers.RandomTranslation(
    (-0.25, 0.25) ,
    (-0.25, 0.25) ,
    fill_mode="reflect",
    interpolation="bilinear",
    seed=seed
)(x)

x = tf.keras.layers.RandomRotation(
    (-0.25, 0.25),
    fill_mode="reflect",
    interpolation="bilinear",
    seed=seed
)(x)

x = tf.keras.layers.RandomContrast(0.2, seed=seed)(x)


# ...
